In [1]:
# ⚙️ Install dependencies
!pip install transformers torchaudio librosa ipywidgets deepface --quiet
!pip install gradio --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 6.5 MB/s eta 0:00:00
  

In [2]:
import io
import numpy as np
import librosa
import torch
from PIL import Image
from IPython.display import display
import ipywidgets as widgets
from transformers import pipeline, Wav2Vec2ForSequenceClassification, Wav2Vec2FeatureExtractor
from deepface import DeepFace


25-04-18 18:33:31 - Directory /root/.deepface has been created
25-04-18 18:33:31 - Directory /root/.deepface/weights has been created


In [3]:
# 🤖 Text Emotion Classifier
text_classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", top_k=1)

# 🎤 Voice Emotion Classifier
voice_model = Wav2Vec2ForSequenceClassification.from_pretrained("audeering/wav2vec2-large-robust-12-ft-emotion-msp-dim")
voice_processor = Wav2Vec2FeatureExtractor.from_pretrained("audeering/wav2vec2-large-robust-12-ft-emotion-msp-dim")

# 🏷️ Labels
voice_labels = ['angry', 'disgust', 'fearful', 'happy', 'neutral', 'sad', 'surprised']


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

...

Device set to use cpu


config.json:   0%|          | 0.00/2.34k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/661M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at audeering/wav2vec2-large-robust-12-ft-emotion-msp-dim and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


preprocessor_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

In [4]:
# Text Emotion Detection
def detect_text_emotion(text):
    result = text_classifier(text)[0][0]
    return result['label'], round(result['score'], 3)

# Voice Emotion Detection
def detect_voice_emotion(audio_path):
    waveform, sr = librosa.load(audio_path, sr=16000)
    inputs = voice_processor(waveform, sampling_rate=16000, return_tensors="pt", padding=True)
    with torch.no_grad():
        logits = voice_model(**inputs).logits
    scores = torch.softmax(logits, dim=1)[0]
    top_index = torch.argmax(scores).item()
    return voice_labels[top_index], round(scores[top_index].item(), 3)

# Face Emotion Detection using DeepFace
def detect_face_emotion(image_bytes):
    img = Image.open(io.BytesIO(image_bytes)).convert("RGB")
    img_np = np.array(img)
    try:
        analysis = DeepFace.analyze(img_np, actions=['emotion'], enforce_detection=False)
        emotion = analysis[0]['dominant_emotion']
        score = analysis[0]['emotion'][emotion]
        return emotion, round(score / 100, 3)
    except:
        return "No face detected", 0.0


In [5]:
input_selector = widgets.Dropdown(
    options=["Select Input Type", "Text", "Face Image", "Voice Audio"],
    value="Select Input Type",
    description="Choose:",
)
display(input_selector)

def handle_input(change):
    if change["new"] == "Text":
        text_box = widgets.Textarea(
            placeholder='Type how you feel...',
            description='Your Feeling:',
            layout=widgets.Layout(width='500px', height='100px')
        )
        display(text_box)

        def on_text_submit(change):
            user_text = text_box.value
            if user_text.strip():
                emotion, conf = detect_text_emotion(user_text)
                print(f"🧠 Emotion: {emotion} (Confidence: {conf})")

        text_box.observe(on_text_submit, names='value')

    elif change["new"] == "Face Image":
        uploader = widgets.FileUpload(accept='image/*', multiple=False)
        display(uploader)

        def on_upload(change):
            if uploader.value:
                file_info = list(uploader.value.values())[0]
                emotion, conf = detect_face_emotion(file_info['content'])
                print(f"📷 Emotion: {emotion} (Confidence: {conf})")

        uploader.observe(on_upload, names='value')

    elif change["new"] == "Voice Audio":
        uploader = widgets.FileUpload(accept='.wav', multiple=False)
        display(uploader)

        def on_upload(change):
            if uploader.value:
                file_info = list(uploader.value.values())[0]
                with open("temp.wav", "wb") as f:
                    f.write(file_info['content'])
                emotion, conf = detect_voice_emotion("temp.wav")
                print(f"🎤 Emotion: {emotion} (Confidence: {conf})")

        uploader.observe(on_upload, names='value')

input_selector.observe(handle_input, names='value')


Dropdown(description='Choose:', options=('Select Input Type', 'Text', 'Face Image', 'Voice Audio'), value='Sel…

Textarea(value='', description='Your Feeling:', layout=Layout(height='100px', width='500px'), placeholder='Typ…

🧠 Emotion: fear (Confidence: 0.625)
🧠 Emotion: fear (Confidence: 0.784)
🧠 Emotion: sadness (Confidence: 0.337)
🧠 Emotion: sadness (Confidence: 0.462)
🧠 Emotion: sadness (Confidence: 0.418)
🧠 Emotion: sadness (Confidence: 0.482)
🧠 Emotion: sadness (Confidence: 0.727)
🧠 Emotion: sadness (Confidence: 0.606)
🧠 Emotion: sadness (Confidence: 0.925)
🧠 Emotion: sadness (Confidence: 0.731)
🧠 Emotion: joy (Confidence: 0.815)
🧠 Emotion: joy (Confidence: 0.769)
🧠 Emotion: joy (Confidence: 0.885)
🧠 Emotion: joy (Confidence: 0.825)
🧠 Emotion: joy (Confidence: 0.81)
🧠 Emotion: joy (Confidence: 0.754)
🧠 Emotion: joy (Confidence: 0.569)
🧠 Emotion: joy (Confidence: 0.513)
🧠 Emotion: joy (Confidence: 0.74)
🧠 Emotion: joy (Confidence: 0.41)
🧠 Emotion: joy (Confidence: 0.652)
🧠 Emotion: joy (Confidence: 0.958)
🧠 Emotion: sadness (Confidence: 0.658)
🧠 Emotion: joy (Confidence: 0.956)
🧠 Emotion: sadness (Confidence: 0.578)
🧠 Emotion: sadness (Confidence: 0.566)
